In [1]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import json, snowflake.connector

# establish the connection to snowflake
ctx = snowflake.connector.connect( 
    **json.load(open('/opt/ich/python-snowflake-defaults.json')))
    
# verify and test if connection is working
try: 
    cs = ctx.cursor() 
    cs.execute('SELECT current_version(), current_role(), current_warehouse()')
    print(cs.fetchone())
finally: 
    cs.close()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://login.microsoftonline.com/be00e2c6-806c-45f8-84c3-3fc99f64b8d3/saml2?SAMLRequest=pZPNctsgFEZfRUPXktCPbZmxnHHjpHEmbdxYaafZYXTlUCNQAFlJn75YjmbSRbLpjoFz4XA%2FmJ0918I7gDZcyRxFAUYeSKZKLnc5ui8u%2FQx5xlJZUqEk5OgFDDqbzwytRUMWrX2Ud%2FDUgrGe20ga0i%2FkqNWSKGq4IZLWYIhlZLP4ekPiAJNGK6uYEuhNyccV1BjQ1hkOJaXhTu%2FR2oaEYdd1QZcESu%2FCGGMc4mnoqCPyaeCf3Z3e4aMQp0feEQ5fv7p95vLUgo%2B0tifIkKuiWPvr202BvMWgeq6kaWvQG9AHzuD%2B7uYkYJyBskk0GeGg3ftGuR4GjeYHakFwuQ%2BMVF0l6B6YqpvWugMCNworKEOhdtz1YLXMUbPnpb7YPMXxd%2FE7mdzWWwppdrW%2Ffih%2BpX%2BiC%2FpT1qsv0XXFRVdGiiHvxxByfAx5ZUwLK3mM1ropHCc%2BHvnxqMAZSRISJcEEpw%2FIW7pouaS2rxz8e4%2Bg5k

('7.17.1', 'ICHT_RG_COVOAM_22016_PROD', 'ICHT_WH_COVOAM_22016_PROD')


True

In [3]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.EPISODES
'''
cur = ctx.cursor().execute(query)
episodes = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [4]:
# Filter for relevant columns
episodes = episodes[['SUBJECT', 'SPELL_IDENTIFIER',  'ADMISSION_DATE_TIME', 'DISCHARGE_DATE_TIME']]

In [5]:
# Import
path = r'data/final_problem_dummies.csv'
problem_dummies = pd.read_csv(path)

In [6]:
# Filter for relevant columns
problem_dummies2 = problem_dummies.iloc[:,:3]

In [7]:
# Filter for where DISCHARGE_DATE_TIME is after ADMISSION_DATE_TIME
episodes = episodes.loc[(episodes['DISCHARGE_DATE_TIME'] >= episodes['ADMISSION_DATE_TIME'] )]
# To date time 
problem_dummies2['PROBLEM_DT_TM'] = pd.to_datetime(problem_dummies2['PROBLEM_DT_TM'])
# Merge on subject
los = pd.merge(problem_dummies2, episodes)

In [8]:
# Filter for where ADMISSION_DATE_TIME is after PROBLEM_DT_TM
los = los.loc[(los['ADMISSION_DATE_TIME'] >= los['PROBLEM_DT_TM'])]
# Drop duplicates
los = los.drop_duplicates()

In [9]:
# Calculate los
los['los_delta'] = los['DISCHARGE_DATE_TIME'] - los['ADMISSION_DATE_TIME']
los['los_days'] = (los['los_delta'] / np.timedelta64(1, 'D')).astype(int)
los['los_hours'] = (los['los_delta'] / np.timedelta64(1, 'h')).astype(int)
def f(row):
    if row['los_days'] > 7:
        val = 1
    else:
        val = 0
    return val
los['los_long'] = los.apply(f, axis=1)

In [11]:
# Save
#los.to_csv('los.csv')